# Make tables: calculate p-values and differential expressions for each cancer 

Create a dataframe with p-value results from t-tests for all proteins (trans proteomics when PTEN has cnv deletions compared to PTEN wildtype). The dataframe also includes the change in medians between deletions and wildtype. Prepare these tables for further analysis by creating csv files.

In [1]:
import pandas as pd
import numpy as np
import re

import cptac
import cptac.pancan as pc
import cptac.utils as u
import plot_utils as p

In [2]:
print('cptac version:', cptac.version())

cptac version: 0.9.1


In [3]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
'''
Params
cancer_object: Object. The loaded cancer data set.
all_prot: List. trans proteins used to get proteomics data. 
gene_in: String. Gene used to get mutation status for the Mutation col.
utils: utils package from cptac.  

Returns a dataframe with trans proteomics and mutation status of a specific gene.
'''

def all_prot_format_df(cancer_object, all_prot, gene_in = 'PTEN', utils = u):
    mut_type = cancer_object.get_genotype_all_vars(gene_in, omics_source = 'washu') # washu has cnv
    if cancer_object.get_cancer_type() not in ('pancanluad'):
        # Keep only tumor samples from proteomics
        prot_and_mutations = cancer_object.join_omics_to_mutations(mutations_genes = [gene_in], 
                                            omics_df_name = 'proteomics', omics_source = 'umich', 
                                            omics_genes = all_prot, tissue_type = 'tumor')
        # Reduce a multiindex 
        if isinstance(prot_and_mutations.keys(), pd.core.indexes.multi.MultiIndex):
            prot_and_mutations = utils.reduce_multiindex(prot_and_mutations, levels_to_drop = 1)
            prot_and_mutations = rename_duplicate_cols(prot_and_mutations) # add '_i' to distinguish
            
        # Merge Mutation column from get_genotype_all_vars (includes cnv) with proteomics
        mut_type = mut_type[['Mutation']] # get mutation column (created in get_genotype_all_vars)
        prot_df = prot_and_mutations.iloc[:,:-4] # Keep only proteomics
        merged = prot_df.join(mut_type)
        
        # Keep only Wildtype and deletion
        compare = ['Wildtype_Tumor','Deletion']
        get = merged['Mutation'].isin(compare)
        mut_wt = merged[get]
    
    
    # Luad has no somatic mutations for PTEN which changes some things
    elif cancer_object.get_cancer_type() in ('pancanluad'): 
        # get_genotype_all_vars add cnv data under the column PTEN
        mut_type = mut_type.drop(columns= gene_in)
        # different code because no somatic mutation data for pten (can't join to somatic mutations)
        omics = cancer_object.join_omics_to_omics(df1_name = 'CNV', df2_name='proteomics', df1_source = 'washu', 
                                                  df2_source = 'umich', genes1= gene_in, genes2= all_prot, 
                                                  tissue_type = 'tumor')
        omics = utils.reduce_multiindex(omics, levels_to_drop = 1, flatten = True)
        omics = rename_duplicate_cols(omics)
        omics = omics.drop(columns='PTEN_washu_CNV')
       
        # Merge Mutation column from get_genotype_all_vars (includes cnv) with proteomics
        merged = omics.join(mut_type) 
        # Keep only No_Mutation (wildtype) and deletion
        compare = ['No_Mutation','Deletion']
        get = merged['Mutation'].isin(compare)
        mut_wt = merged[get]
        mut_wt['Mutation'] = np.where(
            mut_wt['Mutation'] == 'No_Mutation', 'Wildtype_Tumor', 'Deletion')
    
    mut_wt = mut_wt.rename(columns={col: re.sub(r'_umich_proteomics', '', col) for col in mut_wt.columns.tolist()})
    return mut_wt


In [4]:
'''
Params
cancer_object: Object of the loaded cancer data set.
cancer_name: String. Name to add to the created df.
del_wt_df: DataFrame. df of only samples with PTEN deletion and wildtype. 
all_prot_list: List. All proteins in proteomics df. 

Returns a df with the differential expression between proteomics with PTEN del and wt (del - wt). 
'''

def get_change_in_medians_df(cancer_object, cancer_name, del_wt_df, all_prot_list):
    if  cancer_object.get_cancer_type() in ('pancanucec'):
        d = del_wt_df[del_wt_df.Mutation == "Truncation"]
        wt = del_wt_df[del_wt_df.Mutation == "Wildtype_Tumor"]
    else:
        d = del_wt_df[del_wt_df.Mutation == "Deletion"]
        wt = del_wt_df[del_wt_df.Mutation == "Wildtype_Tumor"]
    del_med = d.median()
    wt_med = wt.median()

    med_dict = {}
    
    # Correlation: + is mutant up compared to wt, - is mutant down
    for prot in all_prot_list:
        if prot in del_med.index and prot in wt_med.index:
            dif = del_med[prot] - wt_med[prot]
            med_dict[prot] = dif
        
    df = pd.DataFrame.from_dict(med_dict, orient='index', columns=[cancer_name+'_Median'])
    df = df.reset_index().rename(columns={'index':'Umich_Proteomics'})
    
    return df
        

In [5]:
''' Adds '_i' to duplicate col names where the int i increases by one with each duplicate 
Returns a df with unique column names. '''

def rename_duplicate_cols(df):
    cols = pd.Series(df.columns[:])

    for dup in cols[cols.duplicated()].unique(): 
        cols[cols[cols == dup].index.values.tolist()] = [dup + '_' + str(i) if i != 0 else dup for i in range(sum(cols == dup))]

    # rename the columns with the cols list.
    df.columns=cols
    return df

#  Step 1: Create data frames with p-values and differential expressions

Each cancer needs a data frame containing only samples that have PTEN cnv deletions and PTEN wildtype with trans proteomics. Use wrap_ttest to run t-tests for all genes in the proteomic data frame. Use get_change_in_medians_df to create the data frame with differential expression values. Merge both data frames. 

Load in cancer data sets from cptac. 

In [ ]:
en = pc.PancanUcec()
h = pc.PancanHnscc()
l = pc.PancanLuad()
ls = pc.PancanLscc()
o = pc.PancanOv()
r = pc.PancanCcrcc()
co = pc.PancanCoad()
g = pc.PancanGbm()
b = pc.PancanBrca()

bcm_circular_RNA                                 
Index(['C3L-00006', 'C3L-00008', 'C3L-00032', 'C3L-00084', 'C3L-00090',
       'C3L-00098', 'C3L-00136', 'C3L-00137', 'C3L-00139', 'C3L-00143',
       ...
       'C3L-01282.N', 'C3L-01304.N', 'C3L-01307.N', 'C3L-01311.N',
       'C3N-00333.N', 'C3N-00383.N', 'C3N-00858.N', 'C3N-00866.N',
       'C3N-01003.N', 'C3N-01346.N'],
      dtype='object', name='Patient_ID', length=119)
bcm_transcriptomics
Index(['C3L-00006', 'C3L-00008', 'C3L-00032', 'C3L-00084', 'C3L-00090',
       'C3L-00098', 'C3L-00136', 'C3L-00137', 'C3L-00139', 'C3L-00143',
       ...
       'C3L-01282.N', 'C3L-01304.N', 'C3L-01307.N', 'C3L-01311.N',
       'C3N-00333.N', 'C3N-00383.N', 'C3N-00858.N', 'C3N-00866.N',
       'C3N-01003.N', 'C3N-01346.N'],
      dtype='object', name='Patient_ID', length=119)
broad_transcriptomics
Index(['C3L-00006', 'C3L-00008', 'C3L-00032', 'C3L-00090', 'C3L-00098',
       'C3L-00136', 'C3L-00137', 'C3L-00139', 'C3L-00143', 'C3L-00145',
    

bcm_circular_RNA                                 
Index(['C3L-00977', 'C3L-00987', 'C3L-00994', 'C3L-00995', 'C3L-00997',
       'C3L-00999', 'C3L-01138', 'C3L-01237', 'C3L-02617', 'C3L-02621',
       ...
       'C3N-03876.N', 'C3N-03878.N', 'C3N-03888.N', 'C3N-03889.N',
       'C3N-03928.N', 'C3N-03933.N', 'C3N-04152.N', 'C3N-04277.N',
       'C3N-04278.N', 'C3N-04611.N'],
      dtype='object', name='Patient_ID', length=170)
bcm_transcriptomics
Index(['C3L-00977', 'C3L-00987', 'C3L-00994', 'C3L-00995', 'C3L-00997',
       'C3L-00999', 'C3L-01138', 'C3L-01237', 'C3L-02617', 'C3L-02621',
       ...
       'C3N-03876.N', 'C3N-03878.N', 'C3N-03888.N', 'C3N-03889.N',
       'C3N-03928.N', 'C3N-03933.N', 'C3N-04152.N', 'C3N-04277.N',
       'C3N-04278.N', 'C3N-04611.N'],
      dtype='object', name='Patient_ID', length=170)
broad_transcriptomics
Index(['C3L-00977', 'C3L-00987', 'C3L-00994', 'C3L-00995', 'C3L-00997',
       'C3L-00999', 'C3L-01138', 'C3L-01237', 'C3L-02617', 'C3L-02621',
    

bcm_circular_RNA                                 
Index(['11LU013', '11LU016', '11LU022', '11LU035', 'C3L-00001', 'C3L-00009',
       'C3L-00080', 'C3L-00083', 'C3L-00093', 'C3L-00094',
       ...
       'C3N-02423.N', 'C3N-02424.N', 'C3N-02433.N', 'C3N-02529.N',
       'C3N-02572.N', 'C3N-02582.N', 'C3N-02586.N', 'C3N-02587.N',
       'C3N-02588.N', 'C3N-02729.N'],
      dtype='object', name='Patient_ID', length=213)
bcm_transcriptomics
Index(['11LU013', '11LU016', '11LU022', '11LU035', 'C3L-00001', 'C3L-00009',
       'C3L-00080', 'C3L-00083', 'C3L-00093', 'C3L-00094',
       ...
       'C3N-02423.N', 'C3N-02424.N', 'C3N-02433.N', 'C3N-02529.N',
       'C3N-02572.N', 'C3N-02582.N', 'C3N-02586.N', 'C3N-02587.N',
       'C3N-02588.N', 'C3N-02729.N'],
      dtype='object', name='Patient_ID', length=213)
broad_transcriptomics
Index(['11LU013', '11LU016', '11LU022', '11LU035', 'C3L-00001', 'C3L-00009',
       'C3L-00080', 'C3L-00083', 'C3L-00093', 'C3L-00094',
       ...
       'C3N-02423

bcm_circular_RNA                                 
Index(['C3L-00081', 'C3L-00415', 'C3L-00445', 'C3L-00503', 'C3L-00568',
       'C3L-00603', 'C3L-00904', 'C3L-00923', 'C3L-00927', 'C3L-00965',
       ...
       'C3N-03486.N', 'C3N-03848.N', 'C3N-03851.N', 'C3N-03875.N',
       'C3N-03877.N', 'C3N-03880.N', 'C3N-03882.N', 'C3N-04124.N',
       'C3N-04127.N', 'C3N-04162.N'],
      dtype='object', name='Patient_ID', length=211)
bcm_transcriptomics
Index(['C3L-00081', 'C3L-00415', 'C3L-00445', 'C3L-00503', 'C3L-00568',
       'C3L-00603', 'C3L-00904', 'C3L-00923', 'C3L-00927', 'C3L-00965',
       ...
       'C3N-03486.N', 'C3N-03848.N', 'C3N-03851.N', 'C3N-03875.N',
       'C3N-03877.N', 'C3N-03880.N', 'C3N-03882.N', 'C3N-04124.N',
       'C3N-04127.N', 'C3N-04162.N'],
      dtype='object', name='Patient_ID', length=211)
broad_transcriptomics
Index(['C3L-00081', 'C3L-00415', 'C3L-00445', 'C3L-00503', 'C3L-00568',
       'C3L-00603', 'C3L-00904', 'C3L-00923', 'C3L-00927', 'C3L-00965',
    

bcm_transcriptomics                              
Index(['01OV002', '01OV007', '01OV008', '01OV010', '01OV013', '01OV017',
       '01OV018', '01OV019', '01OV023', '01OV026',
       ...
       '17OV040', '18OV001', '20OV005', '22OV001', '26OV002', '26OV008',
       '26OV009', '26OV010', '26OV011', '26OV013'],
      dtype='object', name='Patient_ID', length=101)
broad_transcriptomics
Index(['01OV002', '01OV013', '01OV017', '01OV019', '01OV030', '01OV039',
       '01OV045', '01OV046', '01OV047', '01OV049', '02OV005', '02OV006',
       '02OV008', '02OV022', '02OV023', '02OV029', '02OV032', '02OV035',
       '02OV036', '02OV040', '02OV041', '02OV042', '02OV044', '02OV045',
       '02OV046', '04OV001', '04OV017', '04OV021', '04OV023', '04OV031',
       '04OV033', '04OV036', '04OV037', '04OV040', '04OV051', '04OV053',
       '04OV054', '04OV055', '04OV057', '11OV002', '11OV009', '13OV003',
       '13OV004', '14OV011', '15OV001', '17OV001', '17OV002', '17OV010',
       '17OV011', '17OV013', '1

bcm_circular_RNA                                 
Index(['C3L-00004', 'C3L-00010', 'C3L-00011', 'C3L-00026', 'C3L-00079',
       'C3L-00088', 'C3L-00096', 'C3L-00097', 'C3L-00103', 'C3L-00183',
       ...
       'C3N-01220.N', 'C3N-01261.N', 'C3N-01361.N', 'C3N-01522.N',
       'C3N-01524.N', 'C3N-01646.N', 'C3N-01648.N', 'C3N-01649.N',
       'C3N-01651.N', 'C3N-01808.N'],
      dtype='object', name='Patient_ID', length=185)
bcm_transcriptomics
Index(['C3L-00004', 'C3L-00010', 'C3L-00011', 'C3L-00026', 'C3L-00079',
       'C3L-00088', 'C3L-00096', 'C3L-00097', 'C3L-00103', 'C3L-00183',
       ...
       'C3N-01220.N', 'C3N-01261.N', 'C3N-01361.N', 'C3N-01522.N',
       'C3N-01524.N', 'C3N-01646.N', 'C3N-01648.N', 'C3N-01649.N',
       'C3N-01651.N', 'C3N-01808.N'],
      dtype='object', name='Patient_ID', length=185)
broad_transcriptomics
Index(['C3L-00004', 'C3L-00010', 'C3L-00011', 'C3L-00026', 'C3L-00079',
       'C3L-00088', 'C3L-00096', 'C3L-00097', 'C3L-00103', 'C3L-00183',
    

In [ ]:
#pc.download('all', redownload = True)

# Gbm

Part 1: Format data frame using all_prot_format_df. This returns a dataframe with proteomics and a Mutation column containing only PTEN cnv deletions and wildtype tumors. 

In [ ]:
gene = 'PTEN'
g_prot = g.get_proteomics(source = 'umich')
g_prot = g_prot.drop(columns = 'PTEN') # cis effect
g_prot_list = list(g_prot.columns.get_level_values(0))

g_del_wt = all_prot_format_df(g, g_prot_list)
g_del_wt.head()

Part 2: Run T-tests. Create a data frame of just significant comparisons and another data frame for all comparisons. A moderately stringent correction is used (FDR_BH).

In [ ]:
g_cols = list(g_del_wt.columns[:-1])

# Get all pvals
g_pval = u.wrap_ttest(g_del_wt, 'Mutation', g_cols, return_all = True, correction_method = 'fdr_bh', mincount = 5)
g_pval = g_pval.rename(columns = {'Comparison': 'Umich_Proteomics','P_Value': 'GBM_P_Value'})

In [ ]:
g_pval

Part 3: Create the differential expression df. (median of tumors with PTEN wildtype - median of tumors with PTEN cnv deletions)

In [ ]:
g_cols = list(g_del_wt[:-1])
g_med = get_change_in_medians_df(g, "GBM", g_del_wt, g_cols)
g_med

Part 4: Merge the p-value and differential expression dfs.

In [ ]:
g_merged = g_pval.merge(g_med, on='Umich_Proteomics', how='outer')
g_merged = g_merged.replace(to_replace = r'_proteomics', value = '', regex = True)

# Create csv
g_merged.to_csv('csv/Single_Cancer/GBM_pval_medians_pancan.csv',index=False)
g_merged.head()

# Repeat for other cancers.

In [ ]:
o = pc.PancanOv()

In [ ]:
cancer_objects = {'OV':o} #{'HNSCC':h, 'LUAD':l, 'LSCC':ls, 'BR':b, 'CO':co, 'OV':o}

In [ ]:
# Luad has no somatic mutations for PTEN which changes some things
#elif cancer_object.get_cancer_type() in ('pancanluad'): 
# get_genotype_all_vars add cnv data under the column PTEN

gene_in = 'PTEN'
all_prot = prot_and_iso

mut_type = o.get_genotype_all_vars(gene_in, omics_source = 'washu') # washu has cnv
mut_type = mut_type.drop(columns= gene_in)
# different code because no somatic mutation data for pten (can't join to somatic mutations)
omics = o.join_omics_to_omics(df1_name = 'CNV', df2_name='proteomics', df1_source = 'washu', 
                                          df2_source = 'umich', genes1= gene_in, genes2= all_prot, 
                                          tissue_type = 'tumor')
omics = utils.reduce_multiindex(omics, levels_to_drop = 1, flatten = True)
omics = rename_duplicate_cols(omics)
omics = omics.drop(columns='PTEN_washu_CNV')

In [ ]:
o.get_CNV('umich')

In [ ]:
# Merge Mutation column from get_genotype_all_vars (includes cnv) with proteomics
merged = omics.join(mut_type)
merged

In [ ]:
import warnings
warnings.filterwarnings('ignore')

gene = 'PTEN'
merged_dfs = {}
for cancer in cancer_objects:
    print(cancer)
    prot = cancer_objects[cancer].get_proteomics(source = 'umich')
    prot = prot.drop(columns = 'PTEN') # cis effect
    
    # Reduce a multiindex 
    if isinstance(prot.keys(), pd.core.indexes.multi.MultiIndex):
        prot = u.reduce_multiindex(prot, levels_to_drop = 1)
    prot_list = list(prot.columns)
    
    # Format df for t-test
    del_wt = all_prot_format_df(cancer_objects[cancer], prot_list)
    
    # Get list of unique cols for t-test and get_change_in_medians
    prot_and_iso = list(del_wt.columns[:-1])

    # Get all pvals
    min_num = 5
    all_pval = u.wrap_ttest(del_wt, 'Mutation', prot_and_iso, return_all = True, 
                            correction_method = 'fdr_bh', mincount = min_num)
    all_pval = all_pval.rename(columns = {'Comparison': 'Umich_Proteomics','P_Value': cancer+'_P_Value'})
    
    # Get change in medians
    delta_median_df = get_change_in_medians_df(cancer_objects[cancer], cancer, del_wt, prot_and_iso)

    # Merge pval_df and delta_median_df
    pval_medians_df = all_pval.merge(delta_median_df, on='Umich_Proteomics', how='outer')
    merged_dfs[cancer] = pval_medians_df # testing purposes
    
    # Create csv
    pval_medians_df.to_csv('csv/Single_Cancer/'+cancer+'_pval_medians_pancan.csv', index=False)
    print(pval_medians_df.head(), '\n')

# Endometrial

The Endometrial data set does not have enough cnv deletions to perform a t-test, however the data set does have enough truncation type mutations (nonsense and frameshift). Different code is needed to create the data frame for Endometrial.

In [ ]:
gene = 'PTEN'
prot = en.get_proteomics(source = 'umich')
prot = u.reduce_multiindex(prot, levels_to_drop = 1)
prot = prot.drop(columns = 'PTEN')
e_prot_list = list(prot.columns)

# Get mutations based on priority filter when a sample has multiple mutations
mut_type = en.get_genotype_all_vars(gene, omics_source = 'washu')

# merge cnv with genotype all mut type
prot_and_mutations = en.join_omics_to_mutations(
    mutations_genes = [gene], omics_df_name = 'proteomics', omics_source='umich', omics_genes = e_prot_list, tissue_type = 'tumor') # drop Normal samples
prot_and_mutations = u.reduce_multiindex(prot_and_mutations, levels_to_drop = 1)

prot_df = prot_and_mutations.iloc[:,:-4] #drop Mutation and Location cols
mut_type = mut_type[['Mutation']] # Get Mutation col that includes CNV
merged = prot_df.join(mut_type) # merge 

# Create Truncation category and keep truncation and wt
compare = ['Wildtype_Tumor','Nonsense_Mutation','Frame_Shift_Ins','Frame_Shift_Del']
get = merged['Mutation'].isin(compare)
trunc_wt = merged[get]
trunc_wt['Mutation'] = np.where(
                trunc_wt['Mutation'] == 'Wildtype_Tumor', 'Wildtype_Tumor', 'Truncation')
trunc_wt = rename_duplicate_cols(trunc_wt)

In [ ]:
e_cols = list(trunc_wt.columns[:-1])

# Get all pvals
e_pval = u.wrap_ttest(trunc_wt, 'Mutation', e_cols, return_all = True, correction_method = 'fdr_bh', mincount = 5)
e_pval = e_pval.rename(columns = {'Comparison': 'Umich_Proteomics','P_Value': 'EC_P_Value'})

Differential expression with adaption for trunctation mutations.

In [ ]:
t = trunc_wt[trunc_wt.Mutation == "Truncation"]
wt = trunc_wt[trunc_wt.Mutation == "Wildtype_Tumor"]
trunc_med = t.median()
wt_med = wt.median()

en_d = {}

for prot in e_cols:
    if prot in trunc_med.index and prot in wt_med.index:
        dif = trunc_med[prot] - wt_med[prot]
        en_d[prot] = dif

en_med = pd.DataFrame.from_dict(en_d, orient='index', columns=['EC_Median'])
en_med = en_med.reset_index().rename(columns={'index':'Umich_Proteomics'})

In [ ]:
e_merged = e_pval.merge(en_med, on='Umich_Proteomics',how='outer')
e_merged = e_merged.replace(to_replace = r'_umich_proteomics', value = '', regex = True)

# Create csv
e_merged.to_csv('csv/Single_Cancer/EC_pval_medians_pancan.csv',index=False)
e_merged.head()

# Get number of significant genes in each cancer

In [ ]:
# Add Gbm and En to merged_dfs dictionary
merged_dfs['GBM'] = g_merged
merged_dfs['EC'] = e_merged

In [ ]:
# Using corrected pvals from wrap_ttest, each cancer's cutoff is 0.05
all_sig = []

for cancer in merged_dfs:
    df = merged_dfs[cancer]
    sig_df = df.loc[df[cancer+'_P_Value'] < 0.05]
    print(cancer, 'sig comparisons:', len(sig_df))
    sig_list = list(sig_df['Umich_Proteomics'])
    all_sig.append(sig_list)
        

flat_list = [item for sublist in all_sig for item in sublist] #change list of lists, to just one list
sig = list(set(flat_list)) # keep only one if gene in multiple cancer sig lists
print('\nNumber of significant comparisons in at least 1 cancer:', len(sig))

# Get a list of significant genes in at least one cancer

In [ ]:
s = pd.Series(sig)
print('Number of significant genes in at least one cancer:', len(s))
s.to_csv('csv/list_sig_one_cancer_pancan.csv', index=False)

# Get a list of significant genes in multiple cancers

In [ ]:
# sig in multiple cancers
mult = list(set(i for i in flat_list if flat_list.count(i) > 1)) # Keep genes sig in more than 1 cancer
print('Number of significant genes in mult cancers:', len(mult))

In [ ]:
m = pd.Series(mult)
m = m.replace(to_replace = r'_proteomics$', value = '', regex = True)
m.to_csv('csv/list_sig_multiple_cancers_pancan.csv', index=False)